In [1]:
# !pip install transformers[torch] accelerate
# !pip install wandb

In [2]:
import wandb

In [4]:
wandb.login(key="4de6103347df6561e7258cdef0ef60bbc1233695", relogin=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
import json 
import pandas as pd 
import os 
import pickle

In [6]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np 

In [7]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from sklearn.metrics import classification_report, precision_recall_curve
from sklearn.preprocessing import LabelEncoder


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt',
            return_attention_mask=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

In [87]:
train_df = pd.read_json("./data/dataset/v2-mini/train.json")

In [88]:
texts = train_df['section_content'].values
labels_text = train_df['tags'].values

In [89]:
label_encoder = LabelEncoder()
# Fit and transform the labels to numeric values
labels = label_encoder.fit_transform(labels_text)

In [90]:
OUTPUT_DIR = "./trained_model/mini"

In [91]:
os.makedirs(OUTPUT_DIR)

FileExistsError: [Errno 17] File exists: './trained_model/mini'

In [92]:
with open(os.path.join(OUTPUT_DIR, "tag.le"), 'wb') as file: 
    pickle.dump(label_encoder, file)

In [93]:
num_labels = train_df.tags.unique().shape[0]

In [94]:
# model_name = 'bert-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Load Model

In [95]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Load Dataset

In [96]:
dataset = TextDataset(texts, labels, tokenizer, max_length=512)

In [98]:
# def compute_metrics(p):
#     preds = torch.tensor(p.predictions)
#     sigmoid_preds = torch.sigmoid(preds)
#     pred_labels = (sigmoid_preds > 0.5).int().numpy()
#     true_labels = p.label_ids
#     pred_labels=np.argmax(pred_labels, axis=1)

#     # print(true_labels, true_labels.shape)
#     # print(pred_labels, pred_labels.shape)
#     # print(rounded_labels, rounded_labels.shape)
#     accuracy = accuracy_score(true_labels, pred_labels)
#     precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='macro')
#     return {
#         'accuracy': accuracy,
#         'precision': precision,
#         'recall': recall,
#         'f1': f1
#     }

## Compute Metrics callback

In [99]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [100]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = accuracy_score(labels, predictions, )
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    f1 = f1_score(labels, predictions, average='macro')
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [101]:

training_args = TrainingArguments(
    output_dir='./results-mini',
    run_name="tag-classifier-v1-mini",
    num_train_epochs=100,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    logging_dir='./logs-mini',
    logging_steps=10,
    learning_rate=2e-5,
    eval_strategy="epoch"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset, 
    compute_metrics=compute_metrics

)

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.869700,0.704090,0.865187,0.036049,0.041667,0.038655
2,0.539800,0.542222,0.868404,0.074056,0.054688,0.058108
3,0.342700,0.478676,0.880952,0.166782,0.136590,0.134395
4,0.456200,0.336363,0.920528,0.430874,0.335987,0.352429
5,0.238600,0.232885,0.951094,0.615482,0.572533,0.556058
6,0.260600,0.173096,0.963964,0.621755,0.681965,0.640715
7,0.155300,0.118446,0.973616,0.733031,0.725082,0.705765
8,0.133500,0.091466,0.981017,0.836339,0.821328,0.805814
9,0.090000,0.077029,0.980373,0.827381,0.852757,0.827418
10,0.163100,0.078804,0.979086,0.825880,0.867370,0.828610


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

KeyboardInterrupt: 

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


In [104]:
# Save the model and tokenizer to disk
model_save_path = './trained_model/mini'
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('./trained_model/mini/tokenizer_config.json',
 './trained_model/mini/special_tokens_map.json',
 './trained_model/mini/vocab.txt',
 './trained_model/mini/added_tokens.json')

In [ ]:
# model = BertForSequenceClassification.from_pretrained(model_save_path)
# tokenizer = BertTokenizer.from_pretrained(model_save_path)

In [107]:
test_df = pd.read_json("./data/dataset/v2-mini/test.json")

In [108]:
test_texts = test_df['section_content'].values
test_labels = test_df['tags'].values

In [111]:
from sklearn.metrics import classification_report


In [112]:
test_dataset = TextDataset(test_texts, test_labels, tokenizer, max_length=512)

# Make predictions on the test set
predictions = trainer.predict(test_dataset)

# Extract the logits and convert to predicted labels
logits = predictions.predictions
predicted_labels = np.argmax(logits, axis=-1)

# Convert numeric labels back to original text labels for comparison
predicted_labels_text = label_encoder.inverse_transform(predicted_labels)
true_labels_text = label_encoder.inverse_transform(test_labels)

# Generate a classification report
report = classification_report(true_labels_text, predicted_labels_text, target_names=label_encoder.classes_)


In [114]:
print(report)

                                    precision    recall  f1-score   support

                  Additional Liens       1.00      0.78      0.88         9
                 Asset Disposition       1.00      0.80      0.89         5
            Compliance Certificate       0.33      0.50      0.40         2
           Consequences of Default       0.71      0.91      0.80        11
                  Event of Default       0.67      1.00      0.80         2
                 Events of Default       0.75      1.00      0.86         3
           Facilities / Instrument       0.75      1.00      0.86         9
                Financial Covenant       0.67      0.29      0.40         7
              Financial Statements       1.00      1.00      1.00         1
                    Governing Laws       1.00      1.00      1.00         7
            Incremental Facilities       0.80      0.80      0.80         5
                     Interest Rate       0.89      0.89      0.89         9
           